# Basic face recognizer using pre trained model

Below the is the implementation of a very basic face recognizer which can identify the face of the person showing on a web cam.
The implementation is inspired by two path breaking papers on facial recognition using deep convoluted neural network, namely FaceNet and DeepFace.

I have used pre trained model Keras-OpenFace which is an open source Keras implementation of the OpenFace (Originally Torch implemented) 

The pretrained model that I have used is by Victor Sy Wang's implementation and was loaded using his code: https://github.com/iwantooxxoox/Keras-OpenFace.

# Import the prerequisite libraries

We will be importing utils.py from https://github.com/iwantooxxoox/Keras-OpenFace/blob/master/utils.py (available with code) which contains utility functions to create the neural network and load the weights assoiated with it.

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils
from keras.applications.vgg16 import VGG16

%load_ext autoreload
%autoreload 2

# np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


# Contructing the neural network model
The model here constructed is based on FaceNet's Inception model.

The implementation of model is available at: https://github.com/iwantooxxoox/Keras-OpenFace

In [2]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.




# Loading the model with pretrained weights

FaceNet is trained by minimizing the triplet loss. I will be  loading a previously trained model. weights are avaiable at https://github.com/iwantooxxoox/Keras-OpenFace in the "weights" folder which is also provided in this source.

This can take a couple of minutes to execute and depends on the speed of your machine.


In [3]:
# Load weights from csv files (which was exported from Openface torch model)
weights = utils.weights
weights_dict = utils.load_weights()

# Set layer weights of the model
for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

## About <font color=blue>image_to_embedding</font> function        
When the model is loaded with pre trained weights, then we can create the **128 dimensional embedding vectors** for all the face images stored in the "images" folder. **"image_to_embedding"** function pass an image to the Inception network to generate the embedding vector.

In [4]:
_model = VGG16(weights='imagenet', include_top = False, input_tensor=myInput)

In [5]:
last_layer = _model.get_layer('block5_pool').output

In [6]:
reshape_layer = Flatten()(last_layer)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)

In [7]:
from keras.models import Model
model = Model(inputs=myInput, output=norm_layer)

J:\programes\Anaconda3\envs\venv\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("no...)`
  


In [8]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)

In [9]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
#     print(x_train.shape)
    embedding = model.predict_on_batch(x_train)
    return embedding


## About <font color=blue>recognize_face</font> function
This function calculate similarity between the captured image and the images that are already been stored. It passes the image to the trained neural network to generate its embedding vector. Which is then compared with all the embedding vectors of the images stored by calculating L2 Euclidean distance. 

If the minimum L2 distance between two embeddings is less than a threshpld (here I have taken the threashhold as .68 (which can be adjusted) then we have a match.

In [10]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.68:
        return str(name)
    else:
        return None

## About <font color=blue>create_input_image_embeddings</font> function
This function generates 128 dimensional image ebeddings of all the images stored in the "images" directory by feed forwarding the images to a trained neural network. It creates a dictionary with key as the name of the face and value as embedding


## About <font color=blue>recognize_faces_in_cam</font> function
This function capture image from the webcam, detect a face in it and crop the image to have a face only, which is then passed to recognize_face function. 

In [11]:
import glob

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

def recognize_faces_in_cam(input_embeddings):
    

    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape

        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            

            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

## Capturing the face image
Following code captures 10 face images of the person. They all are stored in **"images"** folder with the name User_1 to User_10. Select a good captured image from the set of 10 images. Rename it with the name of person and delete rest of them. This image will be used for recognizing the the identity of the person using one shot learning.

In [12]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0   

In [17]:
cam = cv2.VideoCapture(0)

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

count = 0
while(True):
    ret, img = cam.read()
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(img, 1.3, 5)
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite("images/User_" + str(count) + ".jpg", img[y1:y2,x1:x2])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 10: # Take 30 face sample and stop video
         break
cam.release()
cv2.destroyAllWindows()

# Lets run the face recognizer program :-)

In [19]:
input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)


Euclidean distance from 388 is 0.55305946
Euclidean distance from akshay is 0.60899097
Euclidean distance from Jeet is 0.49631983
Euclidean distance from Jeet_with_mask is 0.2697972
Euclidean distance from msd is 0.5253483
Euclidean distance from Pankaj is 0.5163542
Euclidean distance from Rajkumar is 0.53389114
Euclidean distance from SRK is 0.5027082
Euclidean distance from 388 is 0.5357257
Euclidean distance from akshay is 0.61323637
Euclidean distance from Jeet is 0.49211684
Euclidean distance from Jeet_with_mask is 0.31206745
Euclidean distance from msd is 0.5280209
Euclidean distance from Pankaj is 0.51119095
Euclidean distance from Rajkumar is 0.5452818
Euclidean distance from SRK is 0.5064614
Euclidean distance from 388 is 0.5490224
Euclidean distance from akshay is 0.5949873
Euclidean distance from Jeet is 0.5009254
Euclidean distance from Jeet_with_mask is 0.29360533
Euclidean distance from msd is 0.514374
Euclidean distance from Pankaj is 0.5196552
Euclidean distance from Ra

Euclidean distance from 388 is 0.6211175
Euclidean distance from akshay is 0.64335227
Euclidean distance from Jeet is 0.512543
Euclidean distance from Jeet_with_mask is 0.25130484
Euclidean distance from msd is 0.56013316
Euclidean distance from Pankaj is 0.5322225
Euclidean distance from Rajkumar is 0.54977554
Euclidean distance from SRK is 0.5334726
Euclidean distance from 388 is 0.5813261
Euclidean distance from akshay is 0.63179344
Euclidean distance from Jeet is 0.503751
Euclidean distance from Jeet_with_mask is 0.2541201
Euclidean distance from msd is 0.53304005
Euclidean distance from Pankaj is 0.49062446
Euclidean distance from Rajkumar is 0.556914
Euclidean distance from SRK is 0.51880324
Euclidean distance from 388 is 0.58674866
Euclidean distance from akshay is 0.64495045
Euclidean distance from Jeet is 0.50819933
Euclidean distance from Jeet_with_mask is 0.24851324
Euclidean distance from msd is 0.5487307
Euclidean distance from Pankaj is 0.51637924
Euclidean distance from 

Euclidean distance from 388 is 0.62037843
Euclidean distance from akshay is 0.5988918
Euclidean distance from Jeet is 0.4926322
Euclidean distance from Jeet_with_mask is 0.32239863
Euclidean distance from msd is 0.5414969
Euclidean distance from Pankaj is 0.5364799
Euclidean distance from Rajkumar is 0.538471
Euclidean distance from SRK is 0.52646476
Euclidean distance from 388 is 0.62553626
Euclidean distance from akshay is 0.598001
Euclidean distance from Jeet is 0.48824653
Euclidean distance from Jeet_with_mask is 0.30978635
Euclidean distance from msd is 0.5404366
Euclidean distance from Pankaj is 0.5339451
Euclidean distance from Rajkumar is 0.5456006
Euclidean distance from SRK is 0.53772676
Euclidean distance from 388 is 0.6180145
Euclidean distance from akshay is 0.593295
Euclidean distance from Jeet is 0.48310632
Euclidean distance from Jeet_with_mask is 0.30244794
Euclidean distance from msd is 0.53733236
Euclidean distance from Pankaj is 0.5285353
Euclidean distance from Raj

Euclidean distance from 388 is 0.62700546
Euclidean distance from akshay is 0.5979644
Euclidean distance from Jeet is 0.48440576
Euclidean distance from Jeet_with_mask is 0.31012526
Euclidean distance from msd is 0.5428992
Euclidean distance from Pankaj is 0.52870536
Euclidean distance from Rajkumar is 0.5333424
Euclidean distance from SRK is 0.52354914
Euclidean distance from 388 is 0.62123674
Euclidean distance from akshay is 0.58695775
Euclidean distance from Jeet is 0.4731899
Euclidean distance from Jeet_with_mask is 0.3166188
Euclidean distance from msd is 0.53514636
Euclidean distance from Pankaj is 0.521494
Euclidean distance from Rajkumar is 0.52768826
Euclidean distance from SRK is 0.53120726
Euclidean distance from 388 is 0.6248095
Euclidean distance from akshay is 0.58666724
Euclidean distance from Jeet is 0.4783955
Euclidean distance from Jeet_with_mask is 0.32423964
Euclidean distance from msd is 0.53645396
Euclidean distance from Pankaj is 0.5169934
Euclidean distance fro

Euclidean distance from 388 is 0.5124599
Euclidean distance from akshay is 0.48385394
Euclidean distance from Jeet is 0.355043
Euclidean distance from Jeet_with_mask is 0.45934236
Euclidean distance from msd is 0.40940413
Euclidean distance from Pankaj is 0.45723805
Euclidean distance from Rajkumar is 0.4117816
Euclidean distance from SRK is 0.5150993
Euclidean distance from 388 is 0.5127204
Euclidean distance from akshay is 0.48082963
Euclidean distance from Jeet is 0.35379815
Euclidean distance from Jeet_with_mask is 0.44613928
Euclidean distance from msd is 0.41696018
Euclidean distance from Pankaj is 0.4470939
Euclidean distance from Rajkumar is 0.4015941
Euclidean distance from SRK is 0.5114934
Euclidean distance from 388 is 0.51930654
Euclidean distance from akshay is 0.4981173
Euclidean distance from Jeet is 0.3658003
Euclidean distance from Jeet_with_mask is 0.4620736
Euclidean distance from msd is 0.42880672
Euclidean distance from Pankaj is 0.46466634
Euclidean distance from 

...

Euclidean distance from 388 is 0.53501093
Euclidean distance from akshay is 0.5491527
Euclidean distance from Jeet is 0.43833265
Euclidean distance from Jeet_with_mask is 0.51325184
Euclidean distance from msd is 0.51733166
Euclidean distance from Pankaj is 0.5349813
Euclidean distance from Rajkumar is 0.5342147
Euclidean distance from SRK is 0.577637
Euclidean distance from 388 is 0.5309674
Euclidean distance from akshay is 0.5766049
Euclidean distance from Jeet is 0.47810686
Euclidean distance from Jeet_with_mask is 0.5020416
Euclidean distance from msd is 0.52960503
Euclidean distance from Pankaj is 0.5470964
Euclidean distance from Rajkumar is 0.5449286
Euclidean distance from SRK is 0.57524467
Euclidean distance from 388 is 0.51950866
Euclidean distance from akshay is 0.58033544
Euclidean distance from Jeet is 0.49138182
Euclidean distance from Jeet_with_mask is 0.51081854
Euclidean distance from msd is 0.53680384
Euclidean distance from Pankaj is 0.5389367
Euclidean distance from

Euclidean distance from 388 is 0.544531
Euclidean distance from akshay is 0.5833381
Euclidean distance from Jeet is 0.49911553
Euclidean distance from Jeet_with_mask is 0.48093477
Euclidean distance from msd is 0.54519385
Euclidean distance from Pankaj is 0.53330296
Euclidean distance from Rajkumar is 0.558625
Euclidean distance from SRK is 0.5627173
Euclidean distance from 388 is 0.54777646
Euclidean distance from akshay is 0.57901376
Euclidean distance from Jeet is 0.49990055
Euclidean distance from Jeet_with_mask is 0.49721694
Euclidean distance from msd is 0.5426506
Euclidean distance from Pankaj is 0.53737634
Euclidean distance from Rajkumar is 0.55449444
Euclidean distance from SRK is 0.5711481
Euclidean distance from 388 is 0.5490568
Euclidean distance from akshay is 0.6052596
Euclidean distance from Jeet is 0.50568175
Euclidean distance from Jeet_with_mask is 0.4880556
Euclidean distance from msd is 0.5614733
Euclidean distance from Pankaj is 0.5497499
Euclidean distance from R

Euclidean distance from 388 is 0.5244098
Euclidean distance from akshay is 0.575743
Euclidean distance from Jeet is 0.4594057
Euclidean distance from Jeet_with_mask is 0.47879115
Euclidean distance from msd is 0.5410249
Euclidean distance from Pankaj is 0.535365
Euclidean distance from Rajkumar is 0.552049
Euclidean distance from SRK is 0.5658642
Euclidean distance from 388 is 0.5403493
Euclidean distance from akshay is 0.5813518
Euclidean distance from Jeet is 0.46543762
Euclidean distance from Jeet_with_mask is 0.49582237
Euclidean distance from msd is 0.545588
Euclidean distance from Pankaj is 0.53487724
Euclidean distance from Rajkumar is 0.55813694
Euclidean distance from SRK is 0.5710999
Euclidean distance from 388 is 0.54772997
Euclidean distance from akshay is 0.58335257
Euclidean distance from Jeet is 0.48525414
Euclidean distance from Jeet_with_mask is 0.49801248
Euclidean distance from msd is 0.5506413
Euclidean distance from Pankaj is 0.538849
Euclidean distance from Rajkum

Euclidean distance from 388 is 0.52909493
Euclidean distance from akshay is 0.52233654
Euclidean distance from Jeet is 0.4471435
Euclidean distance from Jeet_with_mask is 0.54360497
Euclidean distance from msd is 0.47745162
Euclidean distance from Pankaj is 0.49827895
Euclidean distance from Rajkumar is 0.5431031
Euclidean distance from SRK is 0.57331854
Euclidean distance from 388 is 0.54514575
Euclidean distance from akshay is 0.53579086
Euclidean distance from Jeet is 0.44873893
Euclidean distance from Jeet_with_mask is 0.5490557
Euclidean distance from msd is 0.49712837
Euclidean distance from Pankaj is 0.5093046
Euclidean distance from Rajkumar is 0.54999155
Euclidean distance from SRK is 0.5914292
Euclidean distance from 388 is 0.5411678
Euclidean distance from akshay is 0.53325236
Euclidean distance from Jeet is 0.44296628
Euclidean distance from Jeet_with_mask is 0.54355145
Euclidean distance from msd is 0.4985355
Euclidean distance from Pankaj is 0.51415133
Euclidean distance 

Euclidean distance from 388 is 0.53328586
Euclidean distance from akshay is 0.6061257
Euclidean distance from Jeet is 0.45105976
Euclidean distance from Jeet_with_mask is 0.4920363
Euclidean distance from msd is 0.5290394
Euclidean distance from Pankaj is 0.5027106
Euclidean distance from Rajkumar is 0.5735819
Euclidean distance from SRK is 0.55090034
Euclidean distance from 388 is 0.52839977
Euclidean distance from akshay is 0.60074866
Euclidean distance from Jeet is 0.50625783
Euclidean distance from Jeet_with_mask is 0.49747655
Euclidean distance from msd is 0.55668104
Euclidean distance from Pankaj is 0.5327503
Euclidean distance from Rajkumar is 0.5876862
Euclidean distance from SRK is 0.5821857
Euclidean distance from 388 is 0.55283505
Euclidean distance from akshay is 0.6445627
Euclidean distance from Jeet is 0.53916854
Euclidean distance from Jeet_with_mask is 0.50997233
Euclidean distance from msd is 0.5798146
Euclidean distance from Pankaj is 0.57213634
Euclidean distance fro